In [4]:
from PIL import Image
import torch
from torchvision import models, transforms
import torch.nn as nn


# Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights=None)   # same arch as training
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes (NORMAL, PNEUMONIA)
model.load_state_dict(torch.load("pneu.pth", map_location=device))
model = model.to(device)
model.eval()


# Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

class_names = ["NORMAL", "PNEUMONIA"]  # adjust if different


# Prediction Function
def predict_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img_t = transform(img).unsqueeze(0).to(device)  # add batch dimension
    
    with torch.no_grad():
        outputs = model(img_t)
        _, pred = torch.max(outputs, 1)
    
    return class_names[pred.item()]


C:\Users\Stavya\AppData\Local\Temp\ipykernel_25760\1668419044.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("pneu.pth", map_location=

In [3]:
image_path = r"D:/code/Pneumonia Detection/chest_xray/val/NORMAL/NORMAL2-IM-1430-0001.jpeg"  # 🔹 put your path
print("Prediction:", predict_image(image_path))

Prediction: NORMAL
